## **1. Regroupement en sous-corpus**
Plutôt que de récupérer un tag qui aurait été assigné manuellement pour chaque lien, on va le créer en déconstruisant les URLs et en créant des sous-dossiers correspondant à différents sous-niveaux dans l'arborescence du site.  

À décider : jusqu'à quel niveau on regroupe (ça peut être un peu du cas par cas selon l'organisation du site)

Créer des sous-dossiers à l'intérieur

In [25]:
path = '../03-corpus/2-data/1-fr/'
acteur = "inspq"

from pandas import *

# encoding= 'ISO-8859-1' "utf-8"
with open(path + acteur  + '.csv', encoding='utf-8') as f:
    csv = read_csv(f)[["url", "text"]]

FileNotFoundError: [Errno 2] No such file or directory: '../03-corpus/2-data/1-fr/inspq.csv'

In [26]:
liste = [lien for lien in csv['url'].tolist() if not 'actualites' in lien and not 'propos' in lien and not 'joindre' in lien and not 'communiques' in lien and not 'about' in lien]

liste

['https://www.msss.gouv.qc.ca/repertoires/indicateurs-gestion/indicateur-000009/?&date=DESC&titre=ASC&statut=officiel&critere=statut\n',
 'https://www.msss.gouv.qc.ca/repertoires/indicateurs-gestion/indicateur-000202/?&date=DESC\n',
 'https://www.msss.gouv.qc.ca/inc/documents/ministere/salle-de-presse/fiche-technique-super-clinique-pierre-boucher_2017-11-06.pdf\n',
 'https://www.msss.gouv.qc.ca/ministere/salle-de-presse/communique-3043/\n',
 'https://www.msss.gouv.qc.ca/ministere/salle-de-presse/communique-2857/\n',
 'https://www.msss.gouv.qc.ca/professionnels/documents/comites-d-ethique-de-la-recherche/journees-detude/2017-8e-journee/JECER_2017_P05_Michael-Chasse.pdf\n',
 'https://www.msss.gouv.qc.ca/repertoires/indicateurs-gestion/recherche?page=2&date=ASC\n',
 'https://www.msss.gouv.qc.ca/professionnels/promotion-de-la-sante/saines-habitudes/documents-et-publications/\n',
 'https://www.msss.gouv.qc.ca/repertoires/indicateurs-gestion/indicateur-000023/?&date=DESC&titre=DESC&prog=defi

In [18]:
"""
0 = http/s
1 = //
2 = domaine principal (ex. INSPQ)
3 = 1e sous-niveau
4 = 2e sous-niveau
5 = 3e sous-niveau
etc.
"""

liens_2N = [lien for lien in liste if len(lien.split('/')) > 3]
liens_3N = [lien for lien in liste if len(lien.split('/')) > 4]
liens_4N = [lien for lien in liste if len(lien.split('/')) > 5]

In [22]:
regroupements = {}

for lien in liens_3N:
    tag = lien.split('/')[4]
    if tag in regroupements:
        regroupements[tag].append(lien)
        
    else:
        if len(tag)>1:
            regroupements[tag] = []

tags = list(regroupements.keys())

In [23]:
# On garde seulement les sections qui contiennent au moins 5 sous pages
tags_retenus = [tag for tag in tags if len(regroupements[tag]) >=5]

In [24]:
print('Ça nous donne {} tags.'.format(len(tags_retenus)))
tags_retenus

Ça nous donne 43 tags.


['indicateurs-gestion',
 'documents',
 'salle-de-presse',
 'promotion-de-la-sante',
 'statistiques-donnees-sante-bien-etre',
 'sante-environnementale',
 'soins-et-services',
 'vaccination',
 'dependances',
 'itinerance',
 'aines',
 'acces_info',
 'technologies-de-l-information',
 'documentation-sources-de-donnees-et-indicateurs',
 'perinatalite',
 'zoonoses',
 'ressources',
 'lutte-contre-le-cancer',
 'permis',
 'cancer',
 'covid-19',
 'jeunesse',
 'sante-mentale',
 'medecine-au-quebec',
 'maladies-a-declaration-obligatoire',
 'sante-dentaire',
 'deficiences-et-handicaps',
 'traumatismes-et-traumatologie',
 'ethique',
 'politique-prevention-sante',
 'developpement-durable',
 'connaissances',
 'partenaires-du-systeme',
 'tabagisme-et-vapotage',
 'maladies-infectieuses',
 'alcool-drogues-jeu',
 'statistiques-donnees-services-sante-services-sociaux',
 'maladies-chroniques',
 'securite-civile',
 'images',
 'visites-evaluation',
 'systeme-de-sante-et-de-services-sociaux-en-bref',
 'informat

In [11]:
import os, shutil
from pathlib import Path

# Créer un sous-dossier où on va stocker des nouveaux fichiers CSV correspondant aux sous-corpus
folder_path ='../03-corpus/2-sous-corpus/' + acteur + '/'

Path(folder_path).mkdir(parents=True, exist_ok=True)


for tag in tags_retenus:
    file_name =  acteur + "_" + tag

    # Aller chercher les documents du sous-corpus dans le CSV du corpus principal et les copier dans un nouveau CSV
    data = csv[csv["url"].str.match('.*[\-\_\/]'+tag+'.*')]

    data.to_csv(folder_path + file_name + ".csv")
                    